(compressed-air-label)=

# Compressed Air Production

```{figure} /figures/Compressor.svg
---
name: compressor-flowsheet
---
Abstract model representation of a Compressor.
```

## Introduction

In this example we are investigating the provision of compressed air. Compressed air is often required in technical
systems, e.g. for process control, or as storage medium, i.e. in Compressed Air Energy Storage (CAES). To generate
compressed air a compressor is required, which will (in the best case) compress the air in an isentropic process. The
isentropic process is characterized by the fact, that the entropy does not change.

We can model such a process using CoolProp. For example, we want to compress 1.5 kg/s of air from the ambient state to
a pressure of 10 bar. To calculate the energy consumed in the isentropic process, the energy balance introduced in eq.
{eq}`pipe-flow-energy-balance` can be simplified by removing the heat transfer term:

```{math}
:label: compressor-energy-balance
\dot W = \dot m \cdot \left( h_2 - h_1\right)
```

In [ ]:
from CoolProp.CoolProp import PropsSI


T_amb = 20  # °C
p_amb = 1.01325  # bar
fluid = "air"
dot_m = 1.5

h_amb = PropsSI("H", "T", T_amb + 273.15, "P", p_amb * 1e5, fluid)
s_amb = PropsSI("S", "T", T_amb + 273.15, "P", p_amb * 1e5, fluid)

p_1 = p_amb
T_1 = T_amb

p_2 = 10

h_1 = h_amb
s_1 = s_amb
s_2 = s_1

h_2 = PropsSI("H", "S", s_1, "P", p_2 * 1e5, fluid)

dot_W = dot_m * (h_2 - h_1)

dot_W

We can define the exergy balance for the compressor at temperatures above the ambient temperature in the eqs.
{eq}`compressor-fuel-exergy` and {eq}`compressor-product-exergy`. With that definition it is possible to determine fuel,
product and exergy destruction.

```{math}
:label: compressor-fuel-exergy
\dot E_\text{F} = \dot W
```

```{math}
:label: compressor-product-exergy
\dot E_\text{P} = \dot{E}_\text{out}^\text{PH} - \dot{E}_\text{in}^\text{PH}
```

In [ ]:
from utilities import calc_physical_exergy


exergy_fuel = dot_W

ex_1 = calc_physical_exergy(p_1 * 1e5, h_1, p_amb * 1e5, T_amb + 273.15, fluid)
ex_2 = calc_physical_exergy(p_2 * 1e5, h_2, p_amb * 1e5, T_amb + 273.15, fluid)

exergy_product = dot_m * (ex_2 - ex_1)

exergy_destruction = exergy_fuel - exergy_product

exergy_destruction

First of all, we see that the exergy destruction is zero, which verifies our calculation: Since the isentropic process
is adiabatic and reversible we have an thermodynamically speaking ideal process. Since such a component can only exist
in theory, a reference figure is used to define how close an actual compressor is to the theoretical optimum: the
isentropic efficiency {math}`\eta_\text{s,cmp}` in eq. {eq}`compressor-isentropic-efficiency`:

```{math}
:label: compressor-isentropic-efficiency
\eta_\text{s,cmp} = \frac{h \left( p_2, s_1\right) - h_1}{h_2 - h_1}
```

## Exercises 1

Assume you want to compress the same amount of air to the same pressure as in the example before:

- What power does a compressor with an isentropic efficiency of 85 % consume?
- How large is the exergy destruction?
- How and why does the product exergy change?
- What is the overall process exergy efficiency in both cases?

## Solution 1

To calculate the outlet state of the compressor with the isentropic efficiency of 85 % we have to first reorder the eq.
{eq}`compressor-isentropic-efficiency` in a way, that the unknown outlet enthalpy can be calculated:

```{math}
:label: compressor-isentropic-efficiency-reorder
h_2 = \frac{h \left( p_2, s_1\right) - h_1}{\eta_\text{s,cmp}} + h_1
```

Then we create a function to calculate the enthalpy after isentropic compression {math}`h \left( p_2, s_1\right)` and
call it with the respective parameters.

In [ ]:
def isentropic_outlet_enthalpy(p_1, h_1, p_2, fluid):
    s_1 = PropsSI("S", "P", p_1 , "H", h_1, fluid)
    h_2s = PropsSI("H", "P", p_2, "S", s_1, fluid)
    return h_2s


eta_s = 0.85

h_2_real = (isentropic_outlet_enthalpy(p_1 * 1e5, h_1, p_2 * 1e5, fluid) - h_1) / eta_s + h_1

dot_W_real = dot_m * (h_2_real - h_1)

dot_W_real / dot_W - 1

In [ ]:
from myst_nb import glue
glue("compressor-work-increase", round((dot_W_real / dot_W - 1) * 100, 2), display=False)

We can see how the work required increases by about {glue:text}`compressor-work-increase` %. Now let's have a look at
the exergy analysis. We can calculate the balance with the updated outlet state and have a look at the exergy
destruction:

In [ ]:
exergy_fuel_real = dot_W_real

ex_2_real = calc_physical_exergy(p_2 * 1e5, h_2_real, p_amb * 1e5, T_amb + 273.15, fluid)

exergy_product_real = dot_m * (ex_2_real - ex_1)

exergy_destruction_real = exergy_fuel_real - exergy_product_real

exergy_destruction_real

In [ ]:
glue("compressor-exergy-destruction", round(exergy_destruction_real), display=False)
glue("compressor-work-change", round(dot_W_real - dot_W), display=False)

The exergy destruction is at {glue:text}`compressor-exergy-destruction` W, the change of work transferred by the
compressor (compared to the ideal compressor) is {glue:text}`compressor-work-change` W. This may seem odd at the first
glance. If the power input to the process has increased stronger than the exergy destruction, more product exergy must
be available than before. We can verify this:

In [ ]:
exergy_product_real - exergy_product

Looking at the exergy efficiency we can see, that the isentropic compressor provides 100 % efficiency. The real 
compressor induces exergy destruction, thus cannot have 100 % efficiency.

In [ ]:
epsilon = exergy_product / dot_W
epsilon_real = exergy_product_real / dot_W_real

In [ ]:
epsilon

In [ ]:
epsilon_real

## Overall process product

We have learned how to run energy and exergy balances on a single compressor and found, that a less efficiency 
compressor consumes more power for the provision of the same mass flow at a specific pressure (obviously), but also that
the product exergy increases the less efficient the compressor is.

This seems very counter intuitive as we always provide 1.5 kg/s compressed air at a pressure of 10 bar. However, we have
not yet looked at the outlet temperature of the compressor. We can do that by calculating the temperature at the outlet
pressure and the respective enthalpy.

In [ ]:
T_2_is = PropsSI("T", "P", p_2 * 1e5, "H", h_2, fluid)
T_2_is - 273.15

In [ ]:
T_2 = PropsSI("T", "P", p_2 * 1e5, "H", h_2_real, fluid)
T_2 - 273.15

It becomes clear, that the outlet temperature rises with a lower isentropic efficiecy of the compressor. Obviously, our
total exergy changes, but we want to have a look at how the shares of phyiscal and thermal exergy change.

### Exercise 2

Compare the mechanical and thermal exergy for the compressed air provided by the ideal and the real compressor.

- Why does the mechanical exergy not change?

### Solution 2

We can make use of the exergy splitting to calculate thermal and mechanical exergy and specifically look at the 
mechanical exergy first: It does NOT change.

In [ ]:
from utilities import calc_splitted_physical_exergy


ex_2_ideal_therm, ex_2_ideal_mech = calc_splitted_physical_exergy(p_2 * 1e5, h_2, p_amb * 1e5, T_amb + 273.15, fluid)
ex_2_therm, ex_2_mech = calc_splitted_physical_exergy(p_2 * 1e5, h_2_real, p_amb * 1e5, T_amb + 273.15, fluid)


ex_2_mech - ex_2_ideal_mech

That means the change in product exergy is purely due to a change in thermal exergy:

In [ ]:
dot_m * (ex_2_therm - ex_2_ideal_therm)

In [ ]:
exergy_product_real - exergy_product

What does this mean about the overall process product exergy? In our technical system we want to provide pneumatic 
energy. We cannot make use of the high temperature for this specific reason. Of course, one could use that thermal
exergy to provide heat to other processes, but for now we want to focus on the provison of pressurized air. Thinking
at this, the value of our process lies in the pressure of the air, not in the temperature. Therefore we want to 
investigate more on this by defining a virtual process, which cools the pressurized air back to the ambient temperature
without changing the pressure as illustrated in {numref}`compressor-virtual-cooling-flowsheet`.

```{figure} /figures/Compressor_virtual_cooling.svg
---
name: compressor-virtual-cooling-flowsheet
---
Process representation of the compressor with virtual cooling of the compressed air.
```

## Exercise 3

Balance the virtual process according to the flowsheet above assuming the temperature at point 3 is at the ambient
temperature and the pressure does not change between the states 2 and 3.

- Between the states 1 and 3
  - How does the enthalpy change?
  - How much energy is transferred overall?
  - How do mechanical and thermal exergy change?
- Between the states 2 and 3
  - How much heat is transferred?
  - How much do mechanical and thermal exergy change?
  - How do the heat transfer and the change in thermal exergy connect?

## Solution 3

We have the following constraints in the compressed air system:

```{math}
    :label: eq:compressed-air-information
    T_3 = T_1\\
    p_3 = p_2\\
    p_2 > p_1
```

We have already calculated the enthalpy at the states 1 and 2. Therefore, we calculate the missing value of enthalpy at
state 3 and find the change in enthalpy and the overall energy transfer between the states 1 and 3. Overall, the 
enthalpy changes by about 2.1 kJ/kg and the total energy transfer nets to -3.2 kW.

In [ ]:
T_3 = T_1
p_3 = p_2
h_3 = PropsSI("H", "T", T_3 + 273.15, "P", p_3 * 1e5, "air")
h_3 - h_1

In [ ]:
dot_m * (h_3 - h_1)

For an outside observer not knowing about the specifics of the process, it seems as if the potential of the compressed
air at the state 3 is lower than at the state 1. By calculating the exergy change we can however find, that the 
potential has actually increased.

```{note}
For completeness, we are also calculating the exergy of the state 1, which must be equal to zero, since we are in
ambient state in that point.
```

In [ ]:
ex_1_therm, ex_1_mech = calc_splitted_physical_exergy(p_1 * 1e5, h_1, p_amb * 1e5, T_amb +273.15, "air")
ex_3_therm, ex_3_mech = calc_splitted_physical_exergy(p_3 * 1e5, h_3, p_amb * 1e5, T_amb +273.15, "air")

(ex_3_therm + ex_3_mech) - (ex_1_therm + ex_1_mech)

We find the changes in the individual constituents of the exergy, the thermal exergy does obviously not change, because
we are at ambient temperature in both cases. The mechanical exergy has however increased.

In [ ]:
ex_3_therm - ex_1_therm

In [ ]:
ex_3_mech - ex_1_mech

Next we have a look at the process between the states 2 and 3. First, we identify the amount of heat transferred. For
that we have to make the energy balance of that process.

```{math}
\dot Q_\text{2-3} = \dot m \cdot \left( h_3 - h_2\right)
```

We can compare the amount of heat withdrawn with the change in exergy, which is the exergy destruction in the virutal
cooling process. It becomes apparent, that the exergy destruction is lower than the heat transferred, because heat does
not consist of pure exergy. And we can see, that the exergy destruction is purely caused by destruction of thermal
exergy. The mechanical exergy does not change, because the pressure does not change.

```{math}
\dot E_\text{D,2-3} = \dot m \cdot \left( e_3 - e_2\right)
```

In [ ]:
dot_Q = dot_m * (h_3 - h_2)
dot_E_D = dot_m * (ex_3_therm + ex_3_mech - (ex_2_therm + ex_2_mech))

In [ ]:
dot_Q

In [ ]:
dot_E_D

In [ ]:
ex_3_therm - ex_2_therm

In [ ]:
ex_3_mech - ex_2_mech

## Improving the process

To thermodynamically improve the process (minimizing the exergy destruction) we subdivide our compressor into two
separate stages with intermittent cooling as shown in {numref}`compressor-intermittent-cooling-flowsheet`.

```{figure} /figures/Compressor_intermittent_cooling.svg
---
name: compressor-intermittent-cooling-flowsheet
---
Process representation of a compressor with two stages and intermittent cooling.
```

## Exercise 4

Model the process of

- compressing the ambient air to 3 bar,
- cooling the air back to the ambient temperature and
- compressing the air up to 10 bar.

Assume, that the isentropic efficiency of the two compressor stages is 85 %.

## Solution 4

To model a compressor with intermittent cooling, we first define the overall process fuel exergy and product exergy in
eqs. {eq}`compressor-overall-fuel-exergy` and {eq}`compressor-overall-product-exergy`. Exergy losses, e.g. by leaking
mass flow are not considered. The exergy destruction and exergy efficiency can then be calculated according to the
theory (eqs. {eq}`compressor-overall-exergy-destruction` and {eq}`compressor-overall-exergy-efficiency`).

```{math}
:label: compressor-overall-fuel-exergy
\dot E_\text{F,tot} = \sum \dot W
```

```{math}
:label: compressor-overall-product-exergy
\dot E_\text{P,tot} = \dot m \cdot e_\text{4}
```

```{math}
:label: compressor-overall-exergy-destruction
\dot E_\text{D,tot} = \dot E_\text{F,tot} - dot E_\text{P,tot}
```

```{math}
:label: compressor-overall-exergy-efficiency
\eta_\text{tot} = \frac{dot E_\text{P,tot}}{\dot E_\text{F,tot}}
```

We define our 4 states and calculate the missing properties (all enthalpies):

In [ ]:
eta_s = 0.85

p_amb = 1e5
T_amb = 25 + 273.15

p_1 = 1e5
p_2 = 3e5
p_3 = p_2
p_4 = 10e5

T_1 = 25 + 273.15
T_3 = T_1

h_1 = PropsSI("H", "T", T_1, "P", p_1, "air")
h_2 = (isentropic_outlet_enthalpy(p_1, h_1, p_2, fluid) - h_1) / eta_s + h_1
h_3 = PropsSI("H", "T", T_3, "P", p_3, "air")
h_4 = (isentropic_outlet_enthalpy(p_3, h_3, p_4, fluid) - h_3) / eta_s + h_3

dot_W_1 = dot_m * (h_2 - h_1)
dot_W_2 = dot_m * (h_4 - h_3)
ex_4 = calc_physical_exergy(p_4, h_4, p_amb, T_amb, "air")

E_F = dot_W_1 + dot_W_2
E_P = dot_m * ex_4
E_D = E_F - E_P
epsilon = E_P / E_F

In [ ]:
epsilon

In [ ]:
dot_W_1 + dot_W_2

## Exercise 5

Balance the process with energy and exergy:
  - How does the overall power requirement change?
  - How does the overall exergy destruction change?
  - Why is the overall exergy destruction smaller?

## Solution 5

## Exercise 6
  
6. Build a TESPy model for the same process.
   - Run the exergy analysis.
   - Verify the results of the models.
   - Vary the intermediate pressure and calculate the process exergy efficiency for a pressure range of 2 bar to 9 bar.
     Plot the results.

## Solution 3

## Solution 4

## Solution 5

## Solution 6

- Nur Kompressor vs. Kompressor mit Zwischenkühler:
  - Was ist mit dem MItteldruck
  - Nutzen der Zwischenkühlung?
  - Wem ist die Zwischenkühlung im Vergleich zuzuordnen?

- T,s/h,s-Diagramm
- Bezug Integral v dp?